In [12]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import AES, DES, ARC4, PKCS1_OAEP
from Crypto.Random import get_random_bytes

In [13]:
class Nigma:
    def __init__(self):
        pass

    def makeEnvelope(self, message, sessionKey, initializationVector, operationMode, methodCrypto):
        if methodCrypto == "AES":
            try:
                with open("./envelope/AES/output/cryptogram.bin", "wb") as archive:
                    archive.write(self.encryptAES(message, sessionKey, initializationVector, operationMode))
                with open("./envelope/AES/output/encryptSessionKey.bin", "wb") as archive:
                    archive.write(self.encryptSessionKey(methodCrypto, sessionKey))
                return True
            except:
                return False
        elif methodCrypto == "DES":
            try:
                with open("./envelope/DES/output/cryptogram.bin", "wb") as archive:
                    archive.write(self.encryptDES(message, sessionKey, initializationVector, operationMode))
                with open("./envelope/DES/output/encryptSessionKey.bin", "wb") as archive:
                    archive.write(self.encryptSessionKey(methodCrypto, sessionKey))
                return True
            except:
                return False
        elif methodCrypto == "RC4":
            try:
                self.encryptRC4(sessionKey)
                with open("./envelope/RC4/output/encryptSessionKey.bin", "wb") as archive:
                    archive.write(self.encryptSessionKey(methodCrypto, sessionKey))
                return True
            except:
                return False
        else:
            return False
        
    def openEnvelope(self, encryptSessionKey, initializationVector, operationMode, methodCrypto):
        if methodCrypto == "AES":
            try:
                self.decryptSessionKey(methodCrypto, encryptSessionKey)

                with open("./openEnvelope/AES/process/decryptSessionKey.bin", "rb") as archive:
                    sessionKey = archive.read()
                    
                self.decryptAES(sessionKey, initializationVector, operationMode)
                return True
            except:
                return False
        elif methodCrypto == "DES":
            try:
                self.decryptSessionKey(methodCrypto, encryptSessionKey)

                with open("./openEnvelope/DES/process/decryptSessionKey.bin", "rb") as archive:
                    sessionKey = archive.read()
                    
                self.decryptDES(sessionKey, initializationVector, operationMode)
                return True
            except:
                return False
        elif methodCrypto == "RC4":
            try:
                self.decryptSessionKey(methodCrypto, encryptSessionKey)
                
                with open("./openEnvelope/RC4/process/decryptSessionKey.bin", "rb") as archive:
                        sessionKey = archive.read()

                self.decryptRC4(sessionKey)
                return True
            except:
                return False
        else:
            return False

    def generateRandomKey(self, sizeKey):
        return get_random_bytes(sizeKey)
    
    def generatePairKeys(self, methodCrypto, sizeKey=1024):
        if methodCrypto == "AES":
            key = RSA.generate(sizeKey)
            try:
                with open("./openEnvelope/AES/input/private.pem", "wb") as archive:
                    archive.write(key.export_key())
                with open("./envelope/AES/input/publickey.pem", "wb") as archive:
                    archive.write(key.publickey().export_key())
                return True
            except:
                return False
        elif methodCrypto == "DES":
            key = RSA.generate(sizeKey)
            try:
                with open("./openEnvelope/DES/input/private.pem", "wb") as archive:
                    archive.write(key.export_key())
                with open("./envelope/DES/input/publickey.pem", "wb") as archive:
                    archive.write(key.publickey().export_key())
                return True
            except:
                return False
        elif methodCrypto == "RC4":
            key = RSA.generate(sizeKey)
            try:
                with open("./openEnvelope/RC4/input/private.pem", "wb") as archive:
                    archive.write(key.export_key())
                with open("./envelope/RC4/input/publickey.pem", "wb") as archive:
                    archive.write(key.publickey().export_key())
                return True
            except:
               return False

    def encryptSessionKey(self, methodCrypto, sessionKey:bytes) -> bytes:
        if methodCrypto == "AES":
            with open("./envelope/AES/input/publickey.pem", "rb") as archive:
                recipient_key = RSA.import_key(archive.read())
                cipher_rsa = PKCS1_OAEP.new(recipient_key)
                return cipher_rsa.encrypt(sessionKey)
        elif methodCrypto == "DES":
            with open("./envelope/DES/input/publickey.pem", "rb") as archive:
                recipient_key = RSA.import_key(archive.read())
                cipher_rsa = PKCS1_OAEP.new(recipient_key)
                return cipher_rsa.encrypt(sessionKey)
        elif methodCrypto == "RC4":
            with open("./envelope/RC4/input/publickey.pem", "rb") as archive:
                recipient_key = RSA.import_key(archive.read())
                cipher_rsa = PKCS1_OAEP.new(recipient_key)
                return cipher_rsa.encrypt(sessionKey)
        
    def decryptSessionKey(self, cryptoMethod, encSessionKey):
        if cryptoMethod == "AES":
            with open("./openEnvelope/AES/input/private.pem") as archive:
                privateKey = RSA.import_key(archive.read())
                cipherRSA = PKCS1_OAEP.new(privateKey)
                try:
                    with open("./openEnvelope/AES/process/decryptSessionKey.bin", "wb") as archive:
                        archive.write(cipherRSA.decrypt(encSessionKey))
                    return True
                except:
                    return False
        elif cryptoMethod == "DES":
            with open("./openEnvelope/DES/input/private.pem") as archive:
                privateKey = RSA.import_key(archive.read())
                cipherRSA = PKCS1_OAEP.new(privateKey)
                try:
                    with open("./openEnvelope/DES/process/decryptSessionKey.bin", "wb") as archive:
                        archive.write(cipherRSA.decrypt(encSessionKey))
                    return True
                except:
                    return False
        elif cryptoMethod == "RC4":
            with open("./openEnvelope/RC4/input/private.pem") as archive:
                privateKey = RSA.import_key(archive.read())
                cipherRSA = PKCS1_OAEP.new(privateKey)
                try:
                    with open("./openEnvelope/RC4/process/decryptSessionKey.bin", "wb") as archive:
                        archive.write(cipherRSA.decrypt(encSessionKey))
                    return True
                except:
                    return False

    def encryptAES(self, message, key, initializationVector, modeOperation):
        if modeOperation == AES.MODE_CBC:
            cipher = AES.new(key, modeOperation, initializationVector)
            sizeBlock = AES.block_size
            messagePadded = message + (sizeBlock - len(message) % sizeBlock) * b"\x00"
            return cipher.encrypt(messagePadded)
        elif modeOperation == AES.MODE_ECB:
            cipher = AES.new(key, modeOperation)

    
    def encryptDES(self, message, key, initializationVector, modeOperation):
        if modeOperation == DES.MODE_CBC:
            cipher = DES.new(key, modeOperation, initializationVector)
            blockSize = DES.block_size
            messagePadded = message + (blockSize - len(message) % blockSize) * b"\x00"
            return cipher.encrypt(messagePadded)
        elif modeOperation == DES.MODE_ECB:
            pass

    def encryptRC4(self, key):
        try:
            cipher = ARC4.new(key)
            with open("./message.txt", "rb") as archive:
                with open("./envelope/RC4/output/cryptogram.bin", "wb") as archiveEncrypted:
                    while True:
                        chunk = archive.read(4096)
                        if len(chunk) == 0:
                            break
                        encryptedChunk = cipher.encrypt(chunk)
                        archiveEncrypted.write(encryptedChunk)
            return True
        except:
            return False

    def decryptAES(self, key, initializationVector, modeOperation):
        if modeOperation == AES.MODE_CBC:
            with open("./envelope/AES/output/cryptogram.bin", "rb") as archive:
                cryptogram = archive.read()
            cipher = AES.new(key, modeOperation, initializationVector)
            textLight = cipher.decrypt(cryptogram)
            text = textLight.rstrip(b"\x00")
            with open("./openEnvelope/AES/output/text.txt", "wb") as archive:
                archive.write(text)
        elif modeOperation == AES.MODE_ECB:
            pass
    
    def decryptDES(self, key, initializationVector, modeOperation):
        if modeOperation == DES.MODE_CBC:
            with open("./envelope/DES/output/cryptogram.bin", "rb") as archive:
                cryptogram = archive.read()
            cipherDec = DES.new(key, modeOperation, initializationVector)
            textWithPadded = cipherDec.decrypt(cryptogram)
            text = textWithPadded.rstrip(b"\x00")
            with open("./openEnvelope/DES/output/text.txt", "wb") as archive:
                archive.write(text)
        elif modeOperation == DES.MODE_ECB:
            pass
    
    def decryptRC4(self, key):
        try:
            cipher = ARC4.new(key)
            with open("./envelope/RC4/output/cryptogram.bin", "rb") as archiveEncrypted:
                with open("./openEnvelope/RC4/output/text.txt", "wb") as archiveDecrypted:
                    while True:
                        chunk = archiveEncrypted.read(4096)
                        if len(chunk) == 0:
                            break
                        decryptedChunk = cipher.decrypt(chunk)
                        archiveDecrypted.write(decryptedChunk)
            return True
        except:
            return False

    def __str__(self):
        return "Hi, I'm Nigma!"

Debug Envelope

In [14]:
def main():
    maquina = Nigma()
    sessionKey = maquina.generateRandomKey(8)
    initializationVector = maquina.generateRandomKey(8)
    maquina.generatePairKeys("RC4", 1024)

    with open("./message.txt", "rb") as archive:
        message = archive.read()

    maquina.makeEnvelope(message, sessionKey, initializationVector, None, "RC4")

    with open("./envelope/RC4/output/encryptSessionKey.bin", "rb") as archive:
        encryptSessionKey = archive.read()

    maquina.openEnvelope(encryptSessionKey, initializationVector, None, "RC4")




if __name__ == "__main__":
    main()